# Inferring Coupled Oscillatory Frequencies 

This notebook contains preliminary analysis of a simplified coupled oscillator model. The goal is to simulate oscilattors under different coupling strengths and then estimate how well we can infer the native frequency of the oscillator (e.g. the oscillator without any coupling)

In [1]:
# Load packages
using DifferentialEquations, OrdinaryDiffEq
using Plots, Distributions
using FFTW, Peaks, Images

In [ ]:
# Number of oscilators
n = 50
# Spring Constant
k = 20.0
# Time span of integration
tspan = (0.0, 50000π) # TODO Find the relationship between accuracy and number of observed oscillation -- this sets limits on masses that can be detected in a given time interval
# Sampling time
dt = 0.1
# Mass range
uniform_dist = Uniform(100, 500)
# Sample random integer masses
masses = floor.(rand(uniform_dist, n))

# Calculate frequencies
ω = sqrt.(k ./ masses)



In [3]:
# Solve the equations of motiion over time

#Initial Conditions
x₀ = repeat([0.0], n)  # Uniform initial position
dx₀ = repeat([π / 2], n)# Uniform initial velocity    .* sign.(rand(n) .- 0.5)

# Simple Harmonic Oscillator Problem (no coupling)
# Define the problem
function harmonicoscillator(ddu, du, u, ω, t)
    
    ddu[:] .= -ω.^2 .* u 
end

#Pass to solvers
prob = SecondOrderODEProblem(harmonicoscillator, dx₀, x₀, tspan, ω)
sol = solve(prob, DPRKN6(), saveat=dt);


In [ ]:
# We'll get the "image" current as a single signal,
# To simulate that we take the sum of the individual variables as the signal
# The solver gives us the velocity and positions, but we'll drop the velocity, that's why we start with the (n+1) index
sum_positions = transpose(sum(hcat(sol.u...)[(n+1):(2*n), :], dims=1))
sum_positions


In [ ]:
# We don't want the transient, we just want the long term behavior so we will drop 20% of the signal
start_idx = Int64(round(0.2*length(sum_positions)))


In [ ]:
# We can plot the time domain signal 
plot(sol.t[start_idx:end], sum_positions[start_idx:end])

In [ ]:
# We'll calculate the Fourier Transform of the integrated signal (FFT), and take the absolute value to get real values
k_values = abs.(fft(sum_positions[start_idx:end]))


In [ ]:
# We can plot the spectra
plot(k_values[1:10000])

With the frequency spectra we can calculate the estimated masses directly

In [ ]:
# Find the top of the peaks TODO: This could be more clever, in particular we should probably get the width of the peaks too!
ks = findlocalmaxima(k_values[1:20000])
# ks is of type "CartesianIndex" we need to convert it to Type Float64
ks_float = convert.(Float64, ks)
# With the peaks we can calculate the frequency by multiplying by 2pi ( ω = 2π*f), and dividing by the total integration time (e.g. because it's a discrete system)
infered_ωs = 2π .* ks_float ./ tspan[2]
# From the ω values we can calculate the masses
infered_masses = k ./ (infered_ωs.^2)  

println(unique(infered_masses))
length(infered_masses)

In [ ]:
# We can see what the difference between infered_masses and masses are 
sort(unique(masses)) .- sort(unique(infered_masses))

In [ ]:
# And we can plot them against each other, a straight line means they match up perfectly
scatter(sort(infered_masses)[1:end-1], sort(unique(masses)))